# GEMdataset

> This package will hold the GxExM dataset.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
#| hide
from hybridpredictmaize22.GEMdataset import *

In [ ]:
#| export
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
#| hide
def remove_leapdays(weather):
    """ just a hotfix """
    to_remove = []
    for i in list(set(weather_data['Env'])):
        if (sum(weather_data['Env'] == i)) == 366:
            #get indexes
            to_remove.append(max(list(weather_data.loc[weather_data['Env'] == i].index)))
    return weather_data.drop(to_remove)

In [ ]:
ARGS = {
'test_split':2019,
'path_snps':Path('data/snpCompress/'),
'data_path':Path('data/'),
'path_train_weatherTable':Path('data/Training_Data/4_Training_Weather_Data_2014_2021.csv'),
'path_train_yieldTable':Path('data/Training_Data/1_Training_Trait_Data_2014_2021.csv'),
'snp_compression':'PCS_50',
'batch_size':64
}

test_split 2019
path_snps data/snpCompress
data_path data
path_train_weatherTable data/Training_Data/4_Training_Weather_Data_2014_2021.csv
path_train_yieldTable data/Training_Data/1_Training_Trait_Data_2014_2021.csv
snp_compression PCS_50
batch_size 64


In [ ]:
ARGS.test_split

AttributeError: 'dict' object has no attribute 'test_split'

In [ ]:
test_split = 2019
path_snps = Path('data/snpCompress/')
data_path = Path('data/')
path_train_weatherTable =data_path/'Training_Data/4_Training_Weather_Data_2014_2021.csv'
path_train_yieldTable = data_path/'Training_Data/1_Training_Trait_Data_2014_2021.csv'
snp_compression = 'PCS_50'
batch_size = 64

def setup_data():
    
    snp_data = collect_snps(path_snps/snp_compression) # Read in the SNP profiles
    yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
    weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
    
    yield_data = yield_data[yield_data['Twt_kg_m3'].notnull()] #Remove plots w/ missing yields
    #yield_data = yield_data.reset_index()
    weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
    #removes yield data where no weather data
    setYield = set(yield_data['Env'])
    setWeather = set(weather_data['Env'])
    only_yield = setYield - setWeather
    only_weather = setWeather - setYield
    yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
    #removes yield data where no genotype data
    setSNP = set(snp_data[0])
    setYield = set(yield_data['Hybrid'])
    only_yield = setYield - setSNP
    yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

    weather_data = remove_leapdays(weather_data)
    #weather_data = weather_data.reset_index()
    #yield_data=yield_data.sample(frac=1)
    yield_data = yield_data.reset_index()
    
    #Create a GEM dataset
    gem = GEM(test_split)
    gem.Y = YT(yield_data, test_split)
    gem.W = WT(weather_data, test_split)
    gem.SNP = snp_data
    
    tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
    te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)
    
    tr_dataloader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
    te_dataloader = DataLoader(te_ds, batch_size=batch_size, shuffle=False)
    
    return tr_dataloader, te_dataloader

In [ ]:
snp_data = collect_snps(path_snps/snp_compression) # Read in the SNP profiles
yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
yield_data = yield_data[yield_data['Twt_kg_m3'].notnull()] #Remove plots w/ missing yields
weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
#removes yield data where no weather data
setYield = set(yield_data['Env'])
setWeather = set(weather_data['Env'])
only_yield = setYield - setWeather
only_weather = setWeather - setYield
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
#removes yield data where no genotype data
setSNP = set(snp_data[0])
setYield = set(yield_data['Hybrid'])
only_yield = setYield - setSNP
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

weather_data = remove_leapdays(weather_data)
weather_data = weather_data.reset_index()
#yield_data=yield_data.sample(frac=1)
yield_data = yield_data.reset_index()

#Create a GEM dataset
gem = GEM(test_split)

gem.Y = YT(yield_data, test_split)
gem.W = WT(weather_data, test_split)
gem.SNP = snp_data

tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)

tr_dataloader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
te_dataloader = DataLoader(te_ds, batch_size=batch_size, shuffle=True)

In [ ]:
def init_oz(layer):
    inSize = layer.in_features
    outSize = layer.out_features
    layer.weight = torch.nn.Parameter(torch.tensor(np.array(np.random.choice([-1, 0, 1], size=(outSize,inSize)))).type(torch.float32))
    return layer

In [ ]:
class gModel(nn.Module):
    def __init__(self, inSide,hiddenSize):
        super().__init__()
        #self.in = nn.Linear()
        self.inputLayer = init_oz(nn.Linear(hiddenSize,inSide))
        self.hiddenLayer = init_oz(nn.Linear(hiddenSize,1))

    def forward(self,x):
        x = self.inputLayer(x)
        print(x)
        return x

In [ ]:

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = init_oz(nn.Linear(input_size, hidden_size))
        self.relu = nn.ReLU()
        self.fc2 = init_oz(nn.Linear(hidden_size, output_size))

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    
    
    
# Create an instance of the MLP class


In [ ]:
def unhook(t):
    return t.detach().numpy()

In [ ]:
model = MLP(input_size=100, hidden_size=60, output_size=1)#opt = Optimizer(gw.parameters(),lr=0.001)
opt = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
loss_func = nn.MSELoss()
tr_dataloader, te_dataloader = setup_data()

In [ ]:
fit(1, model, loss_func, opt, tr_dataloader, te_dataloader)

In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=torch.optim.SGD): fc.store_attr()

    def one_batch(self):
        self.xb,self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()

    def calc_stats(self):
        acc = (self.preds.argmax(dim=1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss*n)
        self.ns.append(n)

    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for self.num,self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, sum(self.losses).item()/n, sum(self.accs).item()/n)
    
    def fit(self, n_epochs):
        self.accs,self.losses,self.ns = [],[],[]
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()